## Window functions

- Windowed functions fill the niche between `.groupBy().agg()` and group map UDFs.
- Windows determine which records are used for the application of a function.
- Windowed functions preserve the number of records in the data frame: the results of aggregations are broadcasted to all the rows of the group. 
- `pyspark.sql.window.Window` is the builder class for windows, which are represented by the `WindowSpec` objects. 
- `Window.partitionBy(*columns)` will create partitions for values of the columns. 
- To apply a function in the defined window, the `.over(window)` method of a `Column` is used.
- Window functions are an elegant way to avoid self-joins. 
- Ranking functions rank records based on the value of a field. 
- Windows have an `.orderBy()` method that will sort the records within each window partition.
- `F.rank().over(ordered_window)` will rank the values in the window partition according to the column used in `.orderBy()
- Some ranking functions:
  - `F.rank()` is a nonconsecutive rank: same values result in the same rank, the next value after duplicates is offset by the number of duplicates. 
  - `F.dense_rank()` is a dense rank: ties will still have the same rank, but there will be no skips. 
  - `F.percent_rank()` computes `number_of_records_smaller_than_current / (number_of_records_in_window - 1)`.
  - `F.ntile()` creates an arbitrary number of tiles based on the rank of the data (i.e., quartiles, percentiles). 
  - `F.row_number()` will generate a row number regardless of ties.
- The `.orderBy()` of the window does not have the `ascending` parameter. `Column.desc()` should be used instead. 
- `F.lag()` and `F.lead()` will get shifted values of the column.
- `F.cume_dist()` computes the cumulative density function: `number_of_records_le_current / number_of_records_in_window`. 
- Boundaries of a window are called a *window frame*.
- The boundaries are added to a window definition using `.rowsBetween()` and `.rangeBetween()` methods.
- A *growing window* is a window unbounded on one of the sides: the computation uses a different number of records for each row. A *static window* is a window where both ends are bounded relatively to the current row, i.e. `.rowsBetween(-1, 1)`.
- Spark refers to a record being processed within a window as `Window.currentRow`. The rows before take the values from `-1` to `Window.unboundedPreceeding` at the beginning of the window. Similarly, the rows after take values from `1` to `Window.unboundedFollowing`.
- This can be used to create features that only look at the past and avoid leakage. 
- When the ordering is not defined for a window, an unbounded frame is used by default: all values in the window are included for each row. When ordering is defined, a growing window is used by default: for each row, only the rows up to it are used in the computation. **If partial aggregation is not required, the window must be unordered!**
- *Row windows* look at the position of the rows while *range windows* look at the value of the column used to order. Ranges are useful for working with dates and times. Multiple observations of the given time will break the row window.
- `F.unix_timestamp()` converts a date into integer number of seconds since the Unix epoch. Might be useful in range windows because they only work on numeric values.
- `F.to_date()` creates a date. 
- `.rangeBetween()` is measured from the current row value.
- Series-to-scalar and group-aggregate Pandas UDFs can be used over windows.

In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder.appName("Window functions").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/29 15:28:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

---

In [4]:
df = spark.read.parquet("data/gsod_noaa/")
df = df.filter(F.col("stn").isin([998252, 949110]))  # N = 718
df = df.filter(F.col("year") == 2019)
df = df.select("stn", "year", "mo", "da", "temp")
df.printSchema()

root
 |-- stn: string (nullable = true)
 |-- year: string (nullable = true)
 |-- mo: string (nullable = true)
 |-- da: string (nullable = true)
 |-- temp: double (nullable = true)



In [5]:
df.show(5, truncate=False, vertical=False)

+------+----+---+---+----+
|stn   |year|mo |da |temp|
+------+----+---+---+----+
|949110|2019|11 |23 |54.9|
|998252|2019|04 |18 |44.7|
|998252|2019|06 |15 |53.3|
|949110|2019|11 |26 |54.7|
|949110|2019|11 |01 |69.9|
+------+----+---+---+----+
only showing top 5 rows



In [6]:
a = Window.partitionBy("mo")
b = Window.partitionBy("mo").orderBy("da")
c = Window.partitionBy("mo").orderBy("da").rowsBetween(Window.unboundedPreceding, Window.currentRow)
d = Window.partitionBy("mo").orderBy("da").rowsBetween(Window.currentRow, Window.unboundedFollowing)
e = Window.partitionBy("mo").orderBy("da").rowsBetween(-2, 2)

three_days_sec = 3600 * 24 * 3
f = Window.partitionBy("mo").orderBy("epoch").rangeBetween(-three_days_sec, Window.currentRow)


res = (
    df
    .withColumn("a", F.max("temp").over(a))
    .withColumn("b", F.max("temp").over(b))
    .withColumn("c", F.max("temp").over(c))
    .withColumn("d", F.max("temp").over(d))
    .withColumn("e", F.max("temp").over(e))
    .withColumn("dt", F.to_date(F.concat_ws("-", "year", "mo", "da")))
    .withColumn("epoch", F.unix_timestamp("dt"))
    .withColumn("f", F.max("temp").over(f))
)

res.orderBy("mo", "da").show(33)

+------+----+---+---+----+----+----+----+----+----+----------+----------+----+
|   stn|year| mo| da|temp|   a|   b|   c|   d|   e|        dt|     epoch|   f|
+------+----+---+---+----+----+----+----+----+----+----------+----------+----+
|949110|2019| 01| 01|65.8|75.9|65.8|65.8|75.9|65.8|2019-01-01|1546290000|65.8|
|998252|2019| 01| 01|38.8|75.9|65.8|65.8|75.9|65.8|2019-01-01|1546290000|65.8|
|949110|2019| 01| 02|61.6|75.9|65.8|65.8|75.9|71.3|2019-01-02|1546376400|65.8|
|998252|2019| 01| 02|21.3|75.9|65.8|65.8|75.9|71.3|2019-01-02|1546376400|65.8|
|949110|2019| 01| 03|71.3|75.9|71.3|71.3|75.9|74.1|2019-01-03|1546462800|71.3|
|998252|2019| 01| 03|27.4|75.9|71.3|71.3|75.9|74.1|2019-01-03|1546462800|71.3|
|949110|2019| 01| 04|74.1|75.9|74.1|74.1|75.9|74.1|2019-01-04|1546549200|74.1|
|998252|2019| 01| 04|32.0|75.9|74.1|74.1|75.9|74.1|2019-01-04|1546549200|74.1|
|998252|2019| 01| 05|36.3|75.9|74.1|74.1|75.9|63.9|2019-01-05|1546635600|74.1|
|949110|2019| 01| 05|59.8|75.9|74.1|74.1|75.9|74.1|2